<a href="https://colab.research.google.com/github/zlatovladdka/hse21_hw2/blob/main/hse21_HW2_biopython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Скачиваем данные и устанавливаем программы

In [ ]:
!wget https://raw.githubusercontent.com/vanya-antonov/hse21_hw2/main/data/gms2.lst

In [ ]:
!wget https://raw.githubusercontent.com/vanya-antonov/hse21_hw2/main/data/scaffolds.fasta

In [ ]:
!wget https://raw.githubusercontent.com/vanya-antonov/hse21_hw2/main/data/proteins.fasta

In [ ]:
!wget https://raw.githubusercontent.com/vanya-antonov/hse21_hw2/main/data/scaffolds.hits_from_MIL_1.txt

In [ ]:
!wget https://raw.githubusercontent.com/vanya-antonov/hse21_hw2/main/data/scaffolds.hits_from_SwissProt.txt

## Скачиваем геном близкородственной бактерии T.oleivorans

Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [ ]:
!pip install biopython

## Аннотация генома (пример)

Biopython Tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html

In [ ]:
!head -n 20 gms2.lst

Для примера проаннотируем два коротких скаффолда (scaffold3_cov275 и scaffold9_cov256):



In [ ]:
from Bio import SeqIO

for record in SeqIO.parse("/content/scaffolds.fasta", "fasta"):
  if record.id == 'scaffold3_cov275':
    record_sc3 = record
  if record.id == 'scaffold9_cov256':
    record_sc9 = record

In [ ]:
record_sc3

In [ ]:
record_sc3.id

In [ ]:
len(record_sc3)

In [ ]:
record_sc9.id

In [ ]:
len(record_sc9)

In [ ]:
record_sc3.annotations['molecule_type'] = 'DNA'
record_sc9.annotations['molecule_type'] = 'DNA'

In [ ]:
record_sc3.format("genbank")

In [ ]:
SeqIO.write([record_sc3, record_sc9], "GENOME.gbk", "genbank")

## Добавляем координаты генов

```
SequenceID: scaffold3_cov275
  3535   -    30    1001     972 atypical GTCGAG 5 1
  3536   -    1350    2192     843 native AAAGTG 7 1
  3537   +    2304    2507     204 native GCGGAG 7 1
  3538   -    2554    2751     198 native TAAGTA 7 1
  3539   +    2855    3265     411 native TCGGTC 6 1
```

In [ ]:
from pprint import pprint
from Bio.SeqFeature import SeqFeature, FeatureLocation

sc3_f1 = SeqFeature(FeatureLocation(30, 1001, strand=-1), type="CDS")
sc3_f1.qualifiers['locus_tag'] = ['3535']

sc3_f2 = SeqFeature(FeatureLocation(1350, 2192, strand=-1), type="CDS")
sc3_f2.qualifiers['locus_tag'] = ['3536']

sc3_f3 = SeqFeature(FeatureLocation(2304, 2507, strand=1), type="CDS")
sc3_f3.qualifiers['locus_tag'] = ['3537']

In [ ]:
pprint(vars(sc3_f1))

In [ ]:
record_sc3.features = [sc3_f1, sc3_f2, sc3_f3]

In [ ]:
SeqIO.write([record_sc3, record_sc9], "GENOME.gbk", "genbank")

## Добавляем трансляции генов (аминокислотные посл-ти)

In [ ]:
all_feats = [sc3_f1, sc3_f2, sc3_f3]
for record in SeqIO.parse("proteins.fasta", "fasta"):
  for f in all_feats:
    if record.id == f.qualifiers['locus_tag'][0]:
      f.qualifiers['translation'] = [record.seq]

In [ ]:
record_sc3.features = [sc3_f1, sc3_f2, sc3_f3]

In [ ]:
SeqIO.write([record_sc3, record_sc9], "GENOME.gbk", "genbank")

In [ ]:
pprint(vars(sc3_f1))

## Добавляем функции белков (из бактерии MIL-1)

Часть файла scaffolds.hits_from_MIL_1.txt:
```
3535	lcl|HF680312.1_prot_CCU72326.1_1877	99.054	317	3	0	3	319	1	317	0.0	644
3535	lcl|HF680312.1_prot_CCU72392.1_1943	34.583	240	138	5	85	307	956	1193	2.33e-30	118
3535	lcl|HF680312.1_prot_CCU72759.1_2310	27.667	300	188	8	8	302	709	984	1.02e-24	101
3535	lcl|HF680312.1_prot_CCU70543.1_94	42.857	126	71	1	79	204	798	922	5.63e-24	99.4
3535	lcl|HF680312.1_prot_CCU70690.1_241	26.140	329	205	10	2	303	591	908	6.26e-23	96.7
3535	lcl|HF680312.1_prot_CCU74013.1_3564	38.462	130	76	3	91	219	685	811	4.96e-20	87.8
3535	lcl|HF680312.1_prot_CCU71475.1_1026	36.522	115	71	2	86	200	555	667	2.19e-16	76.6
3535	lcl|HF680312.1_prot_CCU71362.1_913	24.535	269	187	3	52	306	633	899	2.61e-14	70.5
3535	lcl|HF680312.1_prot_CCU70724.1_275	30.534	131	81	3	78	205	1983	2106	6.54e-13	66.2
3535	lcl|HF680312.1_prot_CCU70719.1_270	26.087	115	83	1	90	204	15	127	9.14e-12	58.5
3535	lcl|HF680312.1_prot_CCU72006.1_1557	35.652	115	68	3	87	200	421	530	1.06e-11	62.0
3535	lcl|HF680312.1_prot_CCU72513.1_2064	33.628	113	72	1	91	200	766	878	1.22e-11	62.4
3536	lcl|HF680312.1_prot_CCU72327.1_1878	100.000	280	0	0	1	280	1	280	0.0	570
3536	lcl|HF680312.1_prot_CCU72858.1_2409	27.437	277	181	4	10	272	1	271	1.82e-26	101
3537	lcl|HF680312.1_prot_CCU72328.1_1879	100.000	67	0	0	1	67	1	67	9.75e-44	132
```

In [ ]:
sc3_f1.qualifiers['note'] = ['CCU72326.1']
sc3_f2.qualifiers['note'] = ['CCU72327.1']
sc3_f3.qualifiers['note'] = ['CCU72328.1']

In [ ]:
pprint(sc3_f1.qualifiers)

In [ ]:
mil_1_genome = SeqIO.read("/content/T_oleivorans_MIL_1.gbk", "genbank")

In [ ]:
for  mil_f in mil_1_genome.features:
  if 'protein_id' not in mil_f.qualifiers:
    continue
    
  for f in all_feats:
    if f.qualifiers['note'][0] == mil_f.qualifiers['protein_id'][0]:
      f.qualifiers['product'] = mil_f.qualifiers['product']

In [ ]:
pprint(sc3_f1.qualifiers)

In [ ]:
record_sc3.features = [sc3_f1, sc3_f2, sc3_f3]
SeqIO.write([record_sc3, record_sc9], "GENOME.gbk", "genbank")

## Добавляем функции белков (из БД SwissProt)

Читаем информацию из файла scaffolds.hits_from_SwissProt.txt

In [ ]:
!head scaffolds.hits_from_SwissProt.txt

product = 'Similar to SwissProt protein Y1178_METJA (E-value = 2.95e-28)'

Бонус*

Получить функции белков SwissProt из файла: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

In [ ]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2021-12-16 08:25:46--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625798251 (597M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 596.81M   189MB/s    in 3.2s    

2021-12-16 08:25:49 (185 MB/s) - ‘uniprot_sprot.dat.gz’ saved [625798251/625798251]



In [ ]:
!gzip -d uniprot_sprot.dat.gz

In [ ]:
!grep Q55EH8 uniprot_sprot.dat

AC   Q55EH8;
DR   PaxDb; Q55EH8; -.
DR   InParanoid; Q55EH8; -.
DR   PhylomeDB; Q55EH8; -.
DR   PRO; PR:Q55EH8; -.
